该notebook中的模型，可以做到与biogeme的结果基本一致，包括对于模型的Structural equation，Measurement equation以及Utility function中的参数的估计，对应研究进展20181023。

# Data and Package Prepare

In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import min_max_norm

In [4]:
train_data = pd.read_csv('optif.csv', low_memory=False)
#train_data = pd.read_csv('D:/ProFile/Latent/biogeme/optima_keras.csv', low_memory=False)
random_num = 500
# S=tf.Session()

In [5]:
trainable = True

In [6]:
train_latent = train_data[['ContIncome_0_4000','ContIncome_10000_more', 'ContIncome_4000_6000', 
                           'ContIncome_6000_8000', 'ContIncome_8000_10000', 
                           'age_65_more','haveChildren', 
                           'haveGA', 'highEducation', 'individualHouse', 'male',
                           'moreThanOneBike', 'moreThanOneCar']]

train_PTcom = train_data[['CostPT_HWH', 'CostPT_Other']]
train_CARcom = train_data[['CostCar_HWH', 'CostCar_Other']]

train_PTasc = train_data[['TimePT', 'WaitingTimePT']]
train_CARasc = train_data[['TimeCar']]
train_SMasc = train_data[['distance_km']]

train_indicator = train_data[['Envir01','Envir02','Envir03','Mobil11','Mobil14','Mobil16','Mobil17']]

train_y = train_data[['Choice']]

# Latent Variable Submodel

In [7]:
class LL(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(LL, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = (train_indicator.shape[1], )
        # Create a trainable weight variable for this layer.
        self.sigma = self.add_weight(name='sigma',
                                      shape=shape,
                                     initializer=keras.initializers.Constant(value=np.ones(train_indicator.shape[1])),
#                                      initializer=keras.initializers.Constant(
#                                         value=np.array([1.25,1.12,1.07,1.08,1.05,1.1,1.11])),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(LL, self).build(input_shape)

    def call(self, inputs):
#         error = tf.subtract(inputs[0],inputs[1])
#         se=K.square(error)
        se=K.square(inputs)
        term1 = tf.divide(se, tf.multiply(2., K.square(self.sigma)))
        term2 = K.log(self.sigma)
        term3 = tf.multiply(0.5, K.log(6.283185))
        return tf.add(term1, tf.add(term2, term3))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(LL, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [8]:
class Subtract(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Subtract, self).__init__(**kwargs)

    def build(self, input_shape):

        # Be sure to call this at the end
        super(Subtract, self).build(input_shape)

    def call(self, inputs):
        return tf.cast(tf.subtract(inputs[0],inputs[1]),tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(Subtract, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [9]:
def LL_loss(y_true, y_pred):
    return K.sum(K.sum(K.abs(y_pred - y_true), axis=0))

In [10]:
def multi_indicator_loss(y_true, y_pred):
    n=train_data.shape[0]
    k1,k2 = 14,16
    error2 = K.sum(K.abs(y_pred - y_true), axis=0)
    term1 = tf.multiply(0.5, n-k2)
    term2_1 = tf.multiply(0.5, tf.log(tf.divide(error2, n-k2)))
    term2_2 = tf.multiply(0.5, tf.log(6.283185))
    term2 = tf.multiply(tf.cast(n, tf.float32), tf.add(term2_1, term2_2))
    L2 = tf.add(term1, term2)

    return K.sum(L2)

In [11]:
input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
y_ture = keras.Input(shape = (train_indicator.shape[1],), name = 'input_y_ture')
# car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear',
#                             kernel_initializer=keras.initializers.Constant(
#                                 value=np.array([[ -0.0265 ],
#                                                 [-0.0863],
#                                                 [0.2],
#                                                 [-0.205],
#                                                 [0.541],
#                                                 [-0.077],
#                                                 [0.0182],
#                                                 [ 0.724],
#                                                 [ 0.288],
#                                                 [ 0.0764],
#                                                 [-0.0694],
#                                                 [ 0.293],
#                                                 [-0.68]])),
#                                bias_initializer=keras.initializers.Constant(value=2.27))(input_latent)
car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear')(input_latent)

indicator_ref = keras.layers.Dense(1, name = 'indicator_ref',activation = 'linear',
                               kernel_initializer=keras.initializers.Constant(value=1),
                               bias_initializer=keras.initializers.Constant(value=0))(car_sys)
indicator_other = keras.layers.Dense(train_indicator.shape[1]-1, 
                                     name = 'indicator_other',
                                   kernel_initializer=keras.initializers.RandomUniform(minval=-1., maxval=0., seed=None),
#                                      kernel_initializer=keras.initializers.Constant(
#                                          value=np.array([0.493,-0.706,-0.626,-0.707,-0.542,-0.456])),
#                                      bias_initializer=keras.initializers.Constant(
#                                          value=np.array([2.02,4.64,5.3,4.9,4.79,4.53,])),
                                     activation = 'linear')(car_sys)

indicator = keras.layers.Concatenate(axis=-1)([indicator_ref,indicator_other])

error = Subtract(1)([indicator, y_ture])

loglikelihood = LL(1)(error)

# idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
#                     outputs = indicator)

idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
                    outputs = loglikelihood)

#idmodel.layers[3].trainable = False
idmodel.layers[2].trainable = False
#idmodel.layers[7].trainable = False

idmodel.compile(optimizer=keras.optimizers.Adam(0.01),
              loss = LL_loss)

LLoutput = np.zeros((train_latent.shape[0],train_indicator.shape[1]))

In [12]:
idmodel.evaluate([train_latent, train_indicator], 
          LLoutput, batch_size=train_latent.shape[0])

1545/1545 [==============================] - 5s 3ms/step


119316.046875

In [13]:
idmodel.layers[1].get_weights()

[array([[ 0.5037607 ],
        [-0.5718652 ],
        [ 0.162583  ],
        [ 0.09169263],
        [ 0.23784637],
        [-0.03251243],
        [-0.3057918 ],
        [-0.1295259 ],
        [-0.20398897],
        [ 0.326123  ],
        [ 0.59025836],
        [ 0.3269226 ],
        [-0.156034  ]], dtype=float32), array([0.], dtype=float32)]

In [14]:
idmodel.layers[3].get_weights()

[array([[-0.0546428 , -0.24449205, -0.41301572, -0.53613544, -0.8410039 ,
         -0.8028412 ]], dtype=float32),
 array([0., 0., 0., 0., 0., 0.], dtype=float32)]

In [15]:
idmodel.layers[7].get_weights()

[array([1., 1., 1., 1., 1., 1., 1.], dtype=float32)]

In [16]:
idmodel.fit([train_latent, train_indicator], 
         LLoutput, epochs=8000, batch_size = train_latent.shape[0])

Epoch 1/8000
1545/1545 [==============================] - 0s 120us/step - loss: 119316.0625
Epoch 2/8000
1545/1545 [==============================] - 0s 0us/step - loss: 114855.3281
Epoch 3/8000
1545/1545 [==============================] - 0s 0us/step - loss: 110018.0625
Epoch 4/8000
1545/1545 [==============================] - 0s 10us/step - loss: 105238.8281
Epoch 5/8000
1545/1545 [==============================] - 0s 10us/step - loss: 100663.8750
Epoch 6/8000
1545/1545 [==============================] - 0s 10us/step - loss: 96351.9844
Epoch 7/8000
1545/1545 [==============================] - 0s 10us/step - loss: 92325.5000
Epoch 8/8000
1545/1545 [==============================] - 0s 0us/step - loss: 88588.8125
Epoch 9/8000
1545/1545 [==============================] - 0s 0us/step - loss: 85136.4688
Epoch 10/8000
1545/1545 [==============================] - 0s 0us/step - loss: 81957.3594
Epoch 11/8000
1545/1545 [==============================] - 0s 12us/step - loss: 79037.0859
Epoch 1

Epoch 92/8000
1545/1545 [==============================] - 0s 0us/step - loss: 32389.0234
Epoch 93/8000
1545/1545 [==============================] - 0s 0us/step - loss: 32197.8398
Epoch 94/8000
1545/1545 [==============================] - 0s 10us/step - loss: 32008.6035
Epoch 95/8000
1545/1545 [==============================] - 0s 10us/step - loss: 31821.3184
Epoch 96/8000
1545/1545 [==============================] - 0s 0us/step - loss: 31635.9863
Epoch 97/8000
1545/1545 [==============================] - 0s 10us/step - loss: 31452.6172
Epoch 98/8000
1545/1545 [==============================] - 0s 11us/step - loss: 31271.2227
Epoch 99/8000
1545/1545 [==============================] - 0s 5us/step - loss: 31091.8125
Epoch 100/8000
1545/1545 [==============================] - 0s 10us/step - loss: 30914.4062
Epoch 101/8000
1545/1545 [==============================] - 0s 10us/step - loss: 30739.0195
Epoch 102/8000
1545/1545 [==============================] - 0s 0us/step - loss: 30565.6719
E

Epoch 182/8000
1545/1545 [==============================] - 0s 0us/step - loss: 23216.5703
Epoch 183/8000
1545/1545 [==============================] - 0s 0us/step - loss: 23183.4590
Epoch 184/8000
1545/1545 [==============================] - 0s 10us/step - loss: 23151.0508
Epoch 185/8000
1545/1545 [==============================] - 0s 0us/step - loss: 23119.3184
Epoch 186/8000
1545/1545 [==============================] - 0s 0us/step - loss: 23088.2402
Epoch 187/8000
1545/1545 [==============================] - 0s 0us/step - loss: 23057.7930
Epoch 188/8000
1545/1545 [==============================] - 0s 3us/step - loss: 23027.9531
Epoch 189/8000
1545/1545 [==============================] - 0s 0us/step - loss: 22998.6973
Epoch 190/8000
1545/1545 [==============================] - 0s 0us/step - loss: 22970.0059
Epoch 191/8000
1545/1545 [==============================] - 0s 10us/step - loss: 22941.8574
Epoch 192/8000
1545/1545 [==============================] - 0s 10us/step - loss: 22914.2

Epoch 272/8000
1545/1545 [==============================] - 0s 16us/step - loss: 21522.4238
Epoch 273/8000
1545/1545 [==============================] - 0s 0us/step - loss: 21510.5020
Epoch 274/8000
1545/1545 [==============================] - 0s 10us/step - loss: 21498.6660
Epoch 275/8000
1545/1545 [==============================] - 0s 10us/step - loss: 21486.9121
Epoch 276/8000
1545/1545 [==============================] - 0s 10us/step - loss: 21475.2402
Epoch 277/8000
1545/1545 [==============================] - 0s 10us/step - loss: 21463.6484
Epoch 278/8000
1545/1545 [==============================] - 0s 12us/step - loss: 21452.1406
Epoch 279/8000
1545/1545 [==============================] - 0s 6us/step - loss: 21440.7070
Epoch 280/8000
1545/1545 [==============================] - 0s 0us/step - loss: 21429.3555
Epoch 281/8000
1545/1545 [==============================] - 0s 0us/step - loss: 21418.0820
Epoch 282/8000
1545/1545 [==============================] - 0s 10us/step - loss: 214

Epoch 362/8000
1545/1545 [==============================] - 0s 10us/step - loss: 20702.1699
Epoch 363/8000
1545/1545 [==============================] - 0s 10us/step - loss: 20695.2070
Epoch 364/8000
1545/1545 [==============================] - 0s 10us/step - loss: 20688.2812
Epoch 365/8000
1545/1545 [==============================] - 0s 10us/step - loss: 20681.3906
Epoch 366/8000
1545/1545 [==============================] - 0s 14us/step - loss: 20674.5352
Epoch 367/8000
1545/1545 [==============================] - 0s 2us/step - loss: 20667.7129
Epoch 368/8000
1545/1545 [==============================] - 0s 10us/step - loss: 20660.9258
Epoch 369/8000
1545/1545 [==============================] - 0s 10us/step - loss: 20654.1738
Epoch 370/8000
1545/1545 [==============================] - 0s 10us/step - loss: 20647.4531
Epoch 371/8000
1545/1545 [==============================] - 0s 10us/step - loss: 20640.7676
Epoch 372/8000
1545/1545 [==============================] - 0s 10us/step - loss: 

...

Epoch 7577/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8711
Epoch 7578/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8711
Epoch 7579/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8691
Epoch 7580/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8711
Epoch 7581/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8711
Epoch 7582/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8711
Epoch 7583/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8711
Epoch 7584/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8711
Epoch 7585/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8711
Epoch 7586/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8711
Epoch 7587/8000
1545/1545 [==============================] - 0s 14us/step

Epoch 7666/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8652
Epoch 7667/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8672
Epoch 7668/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8672
Epoch 7669/8000
1545/1545 [==============================] - 0s 5us/step - loss: 17448.8652
Epoch 7670/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8672
Epoch 7671/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8672
Epoch 7672/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8652
Epoch 7673/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8672
Epoch 7674/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8652
Epoch 7675/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8652
Epoch 7676/8000
1545/1545 [==============================] - 0s 0us/step - lo

Epoch 7755/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8613
Epoch 7756/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8613
Epoch 7757/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8613
Epoch 7758/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8633
Epoch 7759/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8633
Epoch 7760/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8633
Epoch 7761/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8613
Epoch 7762/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8633
Epoch 7763/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8613
Epoch 7764/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8613
Epoch 7765/8000
1545/1545 [==============================] - 0s 0us/st

Epoch 7844/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8594
Epoch 7845/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8594
Epoch 7846/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8594
Epoch 7847/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8594
Epoch 7848/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8594
Epoch 7849/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8594
Epoch 7850/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8594
Epoch 7851/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8594
Epoch 7852/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8594
Epoch 7853/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8594
Epoch 7854/8000
1545/1545 [==============================] - 0s 10us/step - 

Epoch 7933/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8574
Epoch 7934/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8574
Epoch 7935/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8574
Epoch 7936/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8555
Epoch 7937/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8555
Epoch 7938/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17448.8574
Epoch 7939/8000
1545/1545 [==============================] - 0s 0us/step - loss: 17448.8574
Epoch 7940/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8574
Epoch 7941/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8574
Epoch 7942/8000
1545/1545 [==============================] - 0s 10us/step - loss: 17448.8574
Epoch 7943/8000
1545/1545 [==============================] - 0s 10us/step 

In [16]:
i = 0
for layer in idmodel.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
indicator_ref 2
indicator_other 3
concatenate 4
input_y_ture 5
subtract 6
ll 7


In [17]:
idmodel.layers[1].get_weights()

[array([[-0.1259092 ],
        [-0.09641296],
        [ 0.3678927 ],
        [-0.34928215],
        [ 0.625567  ],
        [-0.06615467],
        [ 0.05817849],
        [ 0.6222404 ],
        [ 0.21793641],
        [ 0.0993166 ],
        [-0.04281182],
        [ 0.279273  ],
        [-0.61859584]], dtype=float32), array([2.5601623], dtype=float32)]

In [18]:
idmodel.layers[2].get_weights()

[array([[1.]], dtype=float32), array([0.], dtype=float32)]

In [19]:
idmodel.layers[3].get_weights()

[array([[ 0.46806106, -0.77926636, -0.7596789 , -0.70193034, -0.65280664,
         -0.7071629 ]], dtype=float32),
 array([2.0837853, 5.022559 , 5.7847285, 5.0839744, 5.1904716, 5.5095224],
       dtype=float32)]

In [20]:
idmodel.layers[7].get_weights()

[array([1.3653334, 1.1685717, 1.2034935, 1.1288774, 1.1913147, 1.14992  ,
        1.3085434], dtype=float32)]

# Discrete Choice Submodel

计算效用中公共系数部分

V0 alternative specific part

In [17]:
# input [carlover, time_PT_scaled, waitingtimePT]
class V0_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V0_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_time_PT = self.add_weight(name='beta_time_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.72),
                                      # constraints = min_max_norm(min_value=-1.020001, max_value=-1.019999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cl_PT = self.add_weight(name='beta_cl_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=1.37),
                                      # constraints = min_max_norm(min_value=1.929999, max_value=1.930001, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_waiting_time = self.add_weight(name='self.beta_waiting_time',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-0.0275),
                                      # constraints = min_max_norm(min_value=-0.075001, max_value=-0.074999, rate=1.0, axis=0),           
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V0_as, self).build(input_shape)

    def call(self, inputs):
        cl = tf.multiply(inputs[0], self.beta_cl_PT)
        timePT = tf.transpose(tf.multiply(self.beta_time_PT, inputs[1][:,0]))
        waitingtime = tf.transpose(tf.multiply(inputs[1][:,1], self.beta_waiting_time))
        # return tf.transpose(tf.add(timePT, waitingtime))
        return tf.add(timePT, tf.add(waitingtime, cl))
        # return [timePT, waitingtime]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V0_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V1 alternative specific part

In [18]:
# input [carlover, time_CAR_scaled]
class V1_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V1_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.asc_CAR = self.add_weight(name='asc_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=4.66),
                                      # constraints = min_max_norm(min_value=0.740999, max_value=0.741001, rate=1.0, axis=0),
                                      trainable=trainable)
        
        self.beta_time_CAR = self.add_weight(name='beta_time_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-5.83),
                                      # constraints = min_max_norm(min_value=0.482999, max_value=0.483001, rate=1.0, axis=0),          
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V1_as, self).build(input_shape)

    def call(self, inputs):
        # cl = tf.multiply(inputs[0], self.beta_cl_CAR)
        
        timeCAR = tf.multiply(inputs[1], self.beta_time_CAR)
        
        # return tf.transpose(tf.add(timeCAR, self.asc_CAR))
        # return tf.add(tf.add(cl,timeCAR), self.asc_CAR)
        return tf.add(timeCAR, self.asc_CAR)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V1_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V2 alternative specific part

In [19]:
class V2_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V2_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_dist = self.add_weight(name='beta_dist',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.22),
                                      # constraints = min_max_norm(min_value=-4.88001, max_value=-4.87999, rate=1.0, axis=0),   
                                      trainable=trainable)
        self.asc_SM = self.add_weight(name='asc_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.62),
                                      # constraints = min_max_norm(min_value=1.23999, max_value=1.24001, rate=1.0, axis=0), 
                                      trainable=trainable)
        self.beta_cl_SM = self.add_weight(name='beta_cl_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=2.),
                                      # constraints = min_max_norm(min_value=-20.4001, max_value=-20.3999, rate=1.0, axis=0),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V2_as, self).build(input_shape)

    def call(self, inputs):
        cl_SM = tf.multiply(inputs[0], self.beta_cl_SM)
        dist = tf.multiply(inputs[1], self.beta_dist)
        
        return tf.add(tf.add(cl_SM, dist), self.asc_SM)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V2_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [22]:
class TransToRandShape(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(TransToRandShape, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(TransToRandShape, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.zeros((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(TransToRandShape, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [20]:
class Add_Random(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Add_Random, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(Add_Random, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.random_normal((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(Add_Random, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

Model Specification

In [23]:
### input layers
# batch_size = 32

input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
input_PTcom = keras.Input(shape = (train_PTcom.shape[1],), name = 'input_PTcom')
input_CARcom = keras.Input(shape = (train_CARcom.shape[1],), name = 'input_CARcom')
input_PTasc = keras.Input(shape = (train_PTasc.shape[1],), name = 'input_PTasc')
input_CARasc = keras.Input(shape = (train_CARasc.shape[1],), name = 'input_CARasc')
input_SMasc = keras.Input(shape = (train_SMasc.shape[1],), name = 'input_SMasc')

### car lover
idmodel.layers[1].trainable = False
car_sys = idmodel.layers[1](input_latent)
# carlover = keras.layers.Lambda(lambda x: x + tf.random_normal((1505,random_num)), name = 'carlover')(car_sys)
carlover = Add_Random(1,name = 'carlover')(car_sys)

### common coefficients part in utility
utilcommon = keras.layers.Dense(1, name = 'util_common', use_bias=False)
PTcom = TransToRandShape(1)(utilcommon(input_PTcom))
CARcom = TransToRandShape(1)(utilcommon(input_CARcom))


### alternative specific part in utility
# V0
# util_input0 = keras.layers.concatenate([carlover, input_PTasc], axis = -1)
util_V0as = V0_as(1)([carlover, input_PTasc])
V0 = keras.layers.Add(name = 'V0')([util_V0as, PTcom])

# V1
# util_input1 = keras.layers.concatenate([carlover, input_CARasc], axis = -1)
util_V1as = V1_as(1)([carlover, input_CARasc])
V1 = keras.layers.Add(name = 'V1')([util_V1as, CARcom])

# V2
_V2 = V2_as(1)([carlover, input_SMasc])
V2 =keras.layers.Lambda(lambda x: x + tf.zeros((1,random_num)), name = 'V2')(_V2)
### merge
# merged_vector=keras.layers.concatenate([[V0], [V1], [V2]], axis=-1)

prediction =keras.layers.Lambda(lambda x: tf.transpose(tf.reduce_mean(tf.nn.softmax(x, axis = 0),axis=2)), 
                                name = 'prediction')([V0, V1, V2])

model=tf.keras.Model(inputs = [input_latent, input_PTcom, input_CARcom, input_PTasc, input_CARasc, input_SMasc],
                    outputs = prediction)


model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)


In [24]:
model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0])

1545/1545 [==============================] - 0s 317us/step


[14.699870109558105, 0.05760517716407776]

In [25]:
# tf.keras.backend.get_value(util_V1as)
# callbacks = keras.callbacks.TensorBoard(log_dir='./logs')
model.fit([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, epochs=4000, batch_size = train_latent.shape[0])

Epoch 1/4000
1545/1545 [==============================] - 0s 295us/step - loss: 14.7003 - acc: 0.0576
Epoch 2/4000
1545/1545 [==============================] - 0s 20us/step - loss: 14.6946 - acc: 0.0583
Epoch 3/4000
1545/1545 [==============================] - 0s 30us/step - loss: 14.6912 - acc: 0.0589
Epoch 4/4000
1545/1545 [==============================] - 0s 20us/step - loss: 14.6872 - acc: 0.0595
Epoch 5/4000
1545/1545 [==============================] - 0s 30us/step - loss: 14.6857 - acc: 0.0595
Epoch 6/4000
1545/1545 [==============================] - 0s 30us/step - loss: 14.6849 - acc: 0.0595
Epoch 7/4000
1545/1545 [==============================] - 0s 30us/step - loss: 14.6826 - acc: 0.0595
Epoch 8/4000
1545/1545 [==============================] - 0s 29us/step - loss: 14.6805 - acc: 0.0595
Epoch 9/4000
1545/1545 [==============================] - 0s 32us/step - loss: 14.6765 - acc: 0.0595
Epoch 10/4000
1545/1545 [==============================] - 0s 30us/step - loss: 14.6739 - 

Epoch 82/4000
1545/1545 [==============================] - 0s 30us/step - loss: 10.5019 - acc: 0.2712
Epoch 83/4000
1545/1545 [==============================] - 0s 40us/step - loss: 10.4666 - acc: 0.2738
Epoch 84/4000
1545/1545 [==============================] - 0s 40us/step - loss: 10.4412 - acc: 0.2796
Epoch 85/4000
1545/1545 [==============================] - 0s 30us/step - loss: 10.4191 - acc: 0.2822
Epoch 86/4000
1545/1545 [==============================] - 0s 40us/step - loss: 10.4015 - acc: 0.2822
Epoch 87/4000
1545/1545 [==============================] - 0s 40us/step - loss: 10.3859 - acc: 0.2816
Epoch 88/4000
1545/1545 [==============================] - 0s 40us/step - loss: 10.3734 - acc: 0.2841
Epoch 89/4000
1545/1545 [==============================] - 0s 30us/step - loss: 10.3636 - acc: 0.2841
Epoch 90/4000
1545/1545 [==============================] - 0s 30us/step - loss: 10.3602 - acc: 0.2841
Epoch 91/4000
1545/1545 [==============================] - 0s 32us/step - loss: 10

Epoch 162/4000
1545/1545 [==============================] - 0s 30us/step - loss: 9.7254 - acc: 0.3042
Epoch 163/4000
1545/1545 [==============================] - 0s 40us/step - loss: 9.7032 - acc: 0.3042
Epoch 164/4000
1545/1545 [==============================] - 0s 28us/step - loss: 9.6806 - acc: 0.3036
Epoch 165/4000
1545/1545 [==============================] - 0s 30us/step - loss: 9.6547 - acc: 0.3068
Epoch 166/4000
1545/1545 [==============================] - 0s 20us/step - loss: 9.6276 - acc: 0.3074
Epoch 167/4000
1545/1545 [==============================] - 0s 30us/step - loss: 9.6006 - acc: 0.3081
Epoch 168/4000
1545/1545 [==============================] - 0s 20us/step - loss: 9.5698 - acc: 0.3074
Epoch 169/4000
1545/1545 [==============================] - 0s 30us/step - loss: 9.5402 - acc: 0.3087
Epoch 170/4000
1545/1545 [==============================] - 0s 30us/step - loss: 9.5084 - acc: 0.3100
Epoch 171/4000
1545/1545 [==============================] - 0s 30us/step - loss: 9

Epoch 243/4000
1545/1545 [==============================] - 0s 31us/step - loss: 3.1184 - acc: 0.6841
Epoch 244/4000
1545/1545 [==============================] - 0s 28us/step - loss: 3.1049 - acc: 0.6828
Epoch 245/4000
1545/1545 [==============================] - 0s 20us/step - loss: 3.0861 - acc: 0.6835
Epoch 246/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3.0689 - acc: 0.6828
Epoch 247/4000
1545/1545 [==============================] - 0s 20us/step - loss: 3.0587 - acc: 0.6835
Epoch 248/4000
1545/1545 [==============================] - 0s 19us/step - loss: 3.0394 - acc: 0.6841
Epoch 249/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3.0212 - acc: 0.6854
Epoch 250/4000
1545/1545 [==============================] - 0s 30us/step - loss: 3.0055 - acc: 0.6861
Epoch 251/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2.9947 - acc: 0.6861
Epoch 252/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2

Epoch 324/4000
1545/1545 [==============================] - 0s 20us/step - loss: 2.1053 - acc: 0.7120
Epoch 325/4000
1545/1545 [==============================] - 0s 20us/step - loss: 2.1034 - acc: 0.7113
Epoch 326/4000
1545/1545 [==============================] - 0s 20us/step - loss: 2.0917 - acc: 0.7139
Epoch 327/4000
1545/1545 [==============================] - 0s 18us/step - loss: 2.0986 - acc: 0.7133
Epoch 328/4000
1545/1545 [==============================] - 0s 20us/step - loss: 2.0837 - acc: 0.7113
Epoch 329/4000
1545/1545 [==============================] - 0s 20us/step - loss: 2.0775 - acc: 0.7126
Epoch 330/4000
1545/1545 [==============================] - 0s 20us/step - loss: 2.0649 - acc: 0.7107
Epoch 331/4000
1545/1545 [==============================] - 0s 20us/step - loss: 2.0634 - acc: 0.7120
Epoch 332/4000
1545/1545 [==============================] - 0s 20us/step - loss: 2.0581 - acc: 0.7133
Epoch 333/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2

...

Epoch 3612/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6084 - acc: 0.7398
Epoch 3613/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6072 - acc: 0.7327
Epoch 3614/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6077 - acc: 0.7340
Epoch 3615/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6095 - acc: 0.7353
Epoch 3616/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6077 - acc: 0.7346
Epoch 3617/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6062 - acc: 0.7359
Epoch 3618/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6076 - acc: 0.7359
Epoch 3619/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6055 - acc: 0.7372
Epoch 3620/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6085 - acc: 0.7359
Epoch 3621/4000
1545/1545 [==============================] - 0s 20us/step

Epoch 3692/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6063 - acc: 0.7424
Epoch 3693/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6071 - acc: 0.7353
Epoch 3694/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6063 - acc: 0.7333
Epoch 3695/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6076 - acc: 0.7359
Epoch 3696/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6077 - acc: 0.7340
Epoch 3697/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6068 - acc: 0.7359
Epoch 3698/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6075 - acc: 0.7372
Epoch 3699/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6066 - acc: 0.7385
Epoch 3700/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6079 - acc: 0.7411
Epoch 3701/4000
1545/1545 [==============================] - 0s 24us/step

Epoch 3772/4000
1545/1545 [==============================] - 0s 40us/step - loss: 0.6076 - acc: 0.7392
Epoch 3773/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6088 - acc: 0.7366
Epoch 3774/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6069 - acc: 0.7392
Epoch 3775/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6081 - acc: 0.7346
Epoch 3776/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6079 - acc: 0.7327
Epoch 3777/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6086 - acc: 0.7359
Epoch 3778/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6089 - acc: 0.7372
Epoch 3779/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6061 - acc: 0.7320
Epoch 3780/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6086 - acc: 0.7379
Epoch 3781/4000
1545/1545 [==============================] - 0s 11us/step

Epoch 3852/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6070 - acc: 0.7417
Epoch 3853/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6069 - acc: 0.7359
Epoch 3854/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6075 - acc: 0.7398
Epoch 3855/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6061 - acc: 0.7366
Epoch 3856/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6076 - acc: 0.7353
Epoch 3857/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6085 - acc: 0.7385
Epoch 3858/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6071 - acc: 0.7366
Epoch 3859/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6084 - acc: 0.7346
Epoch 3860/4000
1545/1545 [==============================] - 0s 22us/step - loss: 0.6082 - acc: 0.7366
Epoch 3861/4000
1545/1545 [==============================] - 0s 30us/step

Epoch 3932/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6058 - acc: 0.7405
Epoch 3933/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6074 - acc: 0.7359
Epoch 3934/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6071 - acc: 0.7359
Epoch 3935/4000
1545/1545 [==============================] - 0s 18us/step - loss: 0.6091 - acc: 0.7392
Epoch 3936/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6082 - acc: 0.7379
Epoch 3937/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6068 - acc: 0.7366
Epoch 3938/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6070 - acc: 0.7359
Epoch 3939/4000
1545/1545 [==============================] - 0s 20us/step - loss: 0.6076 - acc: 0.7392
Epoch 3940/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6074 - acc: 0.7385
Epoch 3941/4000
1545/1545 [==============================] - 0s 20us/step

In [31]:
lossList=[]
for i in range(100):
    loss_value=model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0],verbose=0)[0]
    lossList.append(loss_value)
print(np.mean(lossList))

0.6071480852365494


In [58]:
model.predict([train_latent[:1], train_PTcom[:1], train_CARcom[:1], train_PTasc[:1], train_CARasc[:1], train_SMasc[:1]])

array([[5.6687373e-01, 4.3312630e-01, 8.0049547e-13]], dtype=float32)

In [39]:
from tensorflow.keras.utils import plot_model
plot_model(idmodel, to_file='linearmodel.png',show_shapes=True)

# Debug part

In [41]:
i = 0
for layer in model.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
carlover 2
input_PTasc 3
input_PTcom 4
input_CARasc 5
input_CARcom 6
input_SMasc 7
v0_as_2 8
util_common 9
v1_as_2 10
v2_as_2 11
V0 12
V1 13
V2 14
prediction 15


In [18]:
model.layers[9].get_weights()

[array([[-1.2084553]], dtype=float32), array([[-0.57047397]], dtype=float32)]

## LatentPart

In [55]:
for i in range(len(output_index)):
    print('{0:15s} {1}'.format(model.layers[output_index[i]].name, model.layers[output_index[i]].get_weights()))

car_sys         [array([[-0.12619334],
       [-0.09635082],
       [ 0.36885747],
       [-0.3496434 ],
       [ 0.6265547 ],
       [-0.06594128],
       [ 0.05845913],
       [ 0.6226478 ],
       [ 0.21824066],
       [ 0.09964096],
       [-0.04249339],
       [ 0.2796297 ],
       [-0.6184652 ]], dtype=float32), array([2.5612953], dtype=float32)]
carlover        []
v0_as_4         [array([[-2.0230172]], dtype=float32), array([[1.5660286]], dtype=float32), array([[-0.02796068]], dtype=float32)]
util_common     [array([[-1.172912]], dtype=float32), array([[-0.47131765]], dtype=float32)]
v1_as_4         [array([[5.2523074]], dtype=float32), array([[-6.434374]], dtype=float32)]
v2_as_4         [array([[-1.5794731]], dtype=float32), array([[-0.4133285]], dtype=float32), array([[1.9687463]], dtype=float32)]
V0              []
V1              []
V2              []
prediction      []


In [47]:
model.layers[9].get_weights()

[array([[-1.392449]], dtype=float32), array([[-0.55060613]], dtype=float32)]